In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dgp import DgpProportion, ExperimentImpact
from design import Sequential

In [8]:
design = Sequential(relative_delta=0.1)
N, d_star = design.fixed_wins_proportion()

N:2922, d*: 106


In [11]:
expt = ExperimentImpact(perc_change=[0,20])
dgp = DgpProportion(prob_success=0.2)
stream_df = dgp.dgp(n_units=N*10, expt_impact=expt) # conservative estimate to reach t+c>=N
stream_df['t_minus_c'] = (((-1)**(stream_df.treatment+1)) * stream_df.outcome).cumsum()
stream_df['t_plus_c'] = stream_df.outcome.cumsum()
stream_df

,unit_id,cluster_id,success_prob,outcome,treatment,t_minus_c,t_plus_c
0,0,0,0.24,0,1,0,0
1,1,0,0.20,0,0,0,0
2,2,0,0.20,0,0,0,0
3,3,0,0.24,1,1,1,1
4,4,0,0.20,0,0,1,1
...,...,...,...,...,...,...,...
8075,8075,0,0.20,1,0,82,1788
8076,8076,0,0.24,0,1,82,1788
8077,8077,0,0.24,0,1,82,1788
8078,8078,0,0.24,0,1,82,1788


In [45]:
p0 = 0.05
p1 = 0.05*(1+design.relative_delta)
((p0/p1)**(T)) * ((1-p0)/(1-p1))**(m-T)

0.05 0.06


In [6]:
effect_found = 0
n_sims = 1_000
for _ in range(n_sims):
	expt = ExperimentImpact(perc_change=[0,0])
	dgp = DgpProportion(prob_success=0.2)
	stream_df = dgp.dgp(n_units=N*10, expt_impact=expt)
	stream_df['t_minus_c'] = (((-1)**(stream_df.treatment+1)) * stream_df.outcome).cumsum()
	stream_df['t_plus_c'] = stream_df.outcome.cumsum()
	effect_found += any((stream_df.t_minus_c==d_star) & (stream_df.t_plus_c<N))
print(effect_found/n_sims)

0.051
